<a id = 'top'></a>

#  A quick-start guide to fine-tune BERT with PyTorch, AutoModel Class, and the Trainer Class
  * A. [Fine-tuned PyTorch Example](#fineTuned)
  * B. [Datasets](#datasetClass)    
      * 1. [GLUE](#glueData)
      * 2. [Exploratory Data Analysis](#EDA)
      * 3. [Model Selection](#modelSelection)
      * 4. [Tokenizer Selection](#tokenizerSelection)
      * 5. [Encode Data](#encodeData)
  * C. [Fine-Tuning](#fineTuning)
      * 1. [Trainer](#trainer)
      * 2. [Train and Evaluate](#trainEval)
      
      

  [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/datasci-w266/2024-fall-main/blob/master/materials/walkthrough_notebooks/bert_as_black_box/HuggingFaceThreeWays_2_Trainer.ipynb)

Hugging Face is a company that offers a library of "transformers" as well as pre-trained language models.  We are going to explore several ways of working with these models at a very high level.  In later classes, when we have covered how a transformer works, we'll come back and look at them at a deeper level.  This tutorial is designed to look at the Huggingface library at the same level as Keras rather at the lower level of TensorFlow.


---

This directory includes three different uses of the HuggingFace Library.  These uses are incompatible with each other so you should only run one at a time and then stop and restart your notebook.  Alternatively, you can copy this notebook to your Google drive file and then open it in a Colab and run it there.  


In [ ]:
!pip install -q transformers

[Return to Top](#top)
 <a id = 'fineTuned'></a>
 # Fine-tuned PyTorch Example

Now let's look at how to train a model.  We'll use a simple data set called COLA that looks at a sentence and says whether it is grammatically correct or not.  We'll use abstract classes that simplify the process of training by consolidating a number of piece under one class.  We'll also use PyTorch which is the native computational graph language used in Hugging Face.  TensorFlow is Google's computational graph language.  There are two reasons to look at this.  First, many models first get put on HuggingFace in PyTorch. Eventually they get ported over to TensorFlow.  Depending on what model you want to use, you may have to run the PyTorch version.  Second, it's important to always be aware of what you're using.  The good news is that HuggingFace has built these models so that the underlying weight parameters can be used across PyTorch and TensorFlow.  It is simply the commands you use to run and manipulate the model that are in PyTorch or TensorFlow.



---


This notebook borrows liberally from https://github.com/huggingface/notebooks/blob/main/examples/text_classification.ipynb which shows how to train a model on a number of GLUE tasks.

In [ ]:
pip install --quiet --upgrade accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 18.7 MB/s eta 0:00:00


[Return to Top](#top)
 <a id = 'datasetClass'></a>
# Datasets



HuggingFace provides a class for the managing datasets.  They also provide a [library of data](https://huggingface.co/datasets) that is accessible via the dataset class.  We'll take advantage of the datasets object in Huggingface to access some well known corpora, specifically GLUE, which contains a set of classification tasks.  It is good for learning how to work with HuggingFace Transformers library and also good for baselines.

In [ ]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

In [ ]:
!pip install -q evaluate
import evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.4 MB/s eta 0:00:00


[Return to Top](#top)
 <a id = 'glueData'></a>
### GLUE

[GLUE](https://gluebenchmark.com/) contains a set of nine different classification tests and a method for aggregating the scores.  The purpose of this dataset is to be able to measure the ability of these large multi-task models. The example below focuses on the COLA task which classifies a sentence as `acceptable` or `unacceptable`.  We'll use the dataset object to look at some of the features of the COLA data.

In [ ]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

In [ ]:
task = "cola"
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
#metric = load_metric('glue', actual_task)
metric = evaluate.load("glue", actual_task)

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/251k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/37.6k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/37.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

[Return to Top](#top)
 <a id = 'EDA'></a>
### Exploratory Data Analysis

Let's look inside the COLA dataset and see what it contains.  We see it has train, validation, and test records.  

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

In [ ]:
dataset["train"][0]

{'sentence': "Our friends won't buy this analysis, let alone the next one we propose.",
 'label': 1,
 'idx': 0}

In [ ]:
#from https://github.com/huggingface/notebooks/blob/main/examples/text_classification.ipynb
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

We can see a random selection of 10 records from the training set and the label associated with each one.  It is a good idea to get to know your data.

In [ ]:
show_random_elements(dataset["train"])

,sentence,label,idx
0,Sylvia jumped the horse over the fence.,acceptable,2185
1,The boat was sunk to collect the insurance.,acceptable,461
2,Spray all the paint onto the wall completely.,acceptable,628
3,The boy swims.,acceptable,4148
4,"The more that you eat, the less that you want.",acceptable,105
5,There are many fish in the sea.,acceptable,7954
6,She was always clad in black.,acceptable,3186
7,John thinks that most depictions of himself are wrong.,acceptable,6340
8,I pushed against the table.,acceptable,2249
9,Jill offered the ball towards Bob.,unacceptable,2053


Each dataset object has a metric object associated with it.  Here you can see the metric object for GLUE.  IT indicates the accepted type(s) of evaluations that can be used with the dataset.  Note that our COLA test uses [Matthews Correlation](https://en.wikipedia.org/wiki/Matthews_correlation_coefficient) rather than accuracy or an F1 score.

In [ ]:
metric  #show the metric object contents

EvaluationModule(name: "glue", module_type: "metric", features: {'predictions': Value(dtype='int64', id=None), 'references': Value(dtype='int64', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = evaluate.load('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=ref

In [ ]:
#quick illustration of having the metric class compute the metric with some fake data
import numpy as np

fake_preds = np.random.randint(0, 2, size=(64,))
fake_labels = np.random.randint(0, 2, size=(64,))
metric.compute(predictions=fake_preds, references=fake_labels)

{'matthews_correlation': -0.055227791305300936}

[Return to Top](#top)
 <a id = 'modelSelection'></a>
### Model Selection


In order to do anything with HuggingFace we need to select a model.  Recall from before that this will indicate a particular architecture with a language and in some cases some task capabilities grafted on to the model.  In this case we want the distilbert model.  This is a variant that runs a smaller architecture and therefore tends to operate faster at the expense of some accuracy in preditiction.  We also specify that we want the base size (as opposed to a large size).  Finally, we want the uncased variant, meaning that all words will be forced to lowercase as part of the tokenization.  If you're lookng for proper nouns then all lowercase text will be problematic.  If you don't case about distinguishing proper nouns from nouns then uncased will most likely help you.

In [ ]:
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

[Return to Top](#top)
 <a id = 'tokenizerSelect'></a>
### Tokenizer Selection

We'll use the AutoTokenizer object because it insures that we get the correct tokenizer given our pre-trained model.  Different models each have their own tokenizer that you can think of as it's own set of learned word embeddings.  You need to instantiate the correct tokenizer for your model.  If we accidentally try to use the FlauBERT tokenizer with DistilBERT, we will have problems even though we may still get poor predictions. The AutoTokenizer object makes sure we use the toeknizer for the model we specified in the model_checkpoint variable above.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


What's the tokenizer doing?  It's taking care of breaking down a sentence into the parts the model can understand and was trained on, as well as a bunch of housekeeping that's needed by the model in order to work properly.  Once we've covered how a transformer works in live session, we'll come back (in week 9) and discuss its various components.  For now, you don't need to understand it in order to make use of it.

In [ ]:
tokenizer("Hello, we only need this one sentence for our task!")

{'input_ids': [101, 7592, 1010, 2057, 2069, 2342, 2023, 2028, 6251, 2005, 2256, 4708, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

The tokenizer converts the incoming words to integer ids that are used to retrieve input word embeddings for the model.  All tokenizers convert words to input ids.  The wrong tokenizer will produce the wrong set of token ids and result in very poor predictions.

---

Below we see some internal scaffolding used to help support all of the GLUE tests and to be able to process single sentence and multi-sentence inputs.

In [ ]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

In [ ]:
sentence1_key, sentence2_key = task_to_keys[task]
if sentence2_key is None:
    print(f"Sentence: {dataset['train'][0][sentence1_key]}")
else:
    print(f"Sentence 1: {dataset['train'][0][sentence1_key]}")
    print(f"Sentence 2: {dataset['train'][0][sentence2_key]}")

Sentence: Our friends won't buy this analysis, let alone the next one we propose.


In [ ]:
sentence1_key

'sentence'

In [ ]:
sentence2_key

[Return to Top](#top)
 <a id = 'encodeData'></a>
### Encode Data


Let's create all of the encoded data for training.  Since we encode it all ahead of time, we can leverage some abstract classes to help us with the training process.

In [ ]:
def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key], truncation=True, padding=True)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)


In [ ]:
preprocess_function(dataset['train'][:5])

{'input_ids': [[101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012, 102], [101, 2028, 2062, 18404, 2236, 3989, 1998, 1045, 1005, 1049, 3228, 2039, 1012, 102, 0, 0, 0, 0, 0], [101, 2028, 2062, 18404, 2236, 3989, 2030, 1045, 1005, 1049, 3228, 2039, 1012, 102, 0, 0, 0, 0, 0], [101, 1996, 2062, 2057, 2817, 16025, 1010, 1996, 13675, 16103, 2121, 2027, 2131, 1012, 102, 0, 0, 0, 0], [101, 2154, 2011, 2154, 1996, 8866, 2024, 2893, 14163, 8024, 3771, 1012, 102, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]}

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/8551 [00:00<?, ? examples/s]

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

Map:   0%|          | 0/1063 [00:00<?, ? examples/s]

[Return to Top](#top)
 <a id = 'fineTuning'></a>
# Fine Tuning

Fine-tuning is the process of taking an already trained model and teaching it to make a new set of predictions.  In HuggingFace we have a large number of models that have been trained to make predictions about language.  These models are then trained a second time, on a new task.  We're going to use a model that was trained on some language tasks and we are going to fine tune it using the COLA data to predict is a sentence in accceptable or unacceptable.

---

In order to do the training we are going to take advantage of some abstract classes from HuggingFace such as the Trainer and the TrainingArguments objects.

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[Return to Top](#top)
 <a id = 'trainer'></a>
### Trainer

We'll use the Trainer object provided by Huggingface to manage the training process for us.  We need to create a structure to hold the arguments for the Trainer class.

In [ ]:
metric_name = "matthews_correlation"

args = TrainingArguments(
    "test-glue",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=False,
    metric_for_best_model=metric_name,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if task != "stsb":
        predictions = np.argmax(predictions, axis=1)
    else:
        predictions = predictions[:, 0]
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
#validation_key = "validation"
validation_key = "validation_mismatched" if task == "mnli-mm" else "validation_matched" if task == "mnli" else "validation"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

[Return to Top](#top)
 <a id = 'trainEval'></a>
### Train and Evaluate

Once we've instantiated the Trainer and Training Model arguments, we can easily train the model based on our new task so it can learn improved weights through back propagation.

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Matthews Correlation
1,0.520200,0.473450,0.421105
2,0.347500,0.456328,0.534378
3,0.231300,0.585422,0.532967
4,0.164000,0.794274,0.539651
5,0.122600,0.820589,0.550741


TrainOutput(global_step=2675, training_loss=0.2658480300190293, metrics={'train_runtime': 244.2285, 'train_samples_per_second': 175.061, 'train_steps_per_second': 10.953, 'total_flos': 501254566711200.0, 'train_loss': 0.2658480300190293, 'epoch': 5.0})

Now that the model is trained, we can feed it a test set to measure its performance.

In [ ]:
trainer.evaluate()

{'eval_loss': 0.8205890655517578,
 'eval_matthews_correlation': 0.550740569901542,
 'eval_runtime': 1.2488,
 'eval_samples_per_second': 835.185,
 'eval_steps_per_second': 52.85,
 'epoch': 5.0}